In [ ]:
pip install --upgrade tensorflow

In [ ]:
import checker
import generator
from IPython.display import display, clear_output
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import importlib
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

print('Tensorflow version:', tf.__version__, '(Minimum expected 2.7.0)')

# Laden von MNIST-Datenset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)


In [ ]:
# Daten normalisieren
x_train_normalized = x_train / 255.0
x_test_normalized = x_test / 255.0
x_train = x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)


### Vorverarbeitung/ Preprocessing
Im Gegensatz zum Standard-Netz wurde bei diesem neuronalen Netz eine Vorverarbeitung bzw. Preprocessing der Labels hinzugefügt. Dies hat folgenden Grund:
Die Idee ist es, die Daten vor dem Training für das Modell so vorzubereiten, dass das Modell effizienter mit den Daten arbeiten kann. Um dies zu erreichen wurde das Label-Format in einen binären Vektor mit der Länge 10 umgewandelt. Die Funktion One-Hot-Encoding wandelt dafür die Ziffern der Original-Labels in Vektoren um, die nur an der Stelle, die der Ziffer entspricht, eine Eins haben und sonst nur Nullen enthalten. Beispiel:
#
•	Original-Label: 8
#
•	One-Hot-Encoding: [0,0,0,0,0,0,0,0,1,0]
#
In dieser Form sind die Labels besser für die Verarbeitung durch das neuronale Netzwerk geeignet, was zur Optimierung des Netzwerks beiträgt, wie auch an den Ergebnissen deutlich beobachtet werden kann.


In [ ]:
# One-Hot-Encoding der Labels
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)


### Modellstruktur und -architektur
Des Weiteren wurde das neuronale Netz in der Architektur angepasst. Die Modelldefiniton zeigt den Aufbau des Netzwerks. Aus Gründen der Optimierung wurde das sequentielle Modell aus folgenden Schichten aufgebaut:
#
•	Conv2D-Schichten (Convolutional Layer):
Die Schichten führen eine Faltung auf dem Eingangsbild durch, um Merkmale aus den zweidimensionalen Eingangsbildern zu erhalten. Mit der Anzahl der Filter (gewählt wurden 32 bzw. 64) kann eingestellt werden, wie viele Merkmale die Schicht aus den Bildern erxtrahiert und im Training lernt. Im Beispiel des MNIST-Datensatzes können z.B. Kanten (z.B. Eck der Sieben) oder Muster (z.B. Kringel der Sechs) erkannt werden. Das steigert die Leistung des Netzwerks bei der Erkennung der Ziffern, da diese Muster u.a. unabhängig von der Position im Bild wiedererkannt werden können. Die Aktivierung erfolgt über Rectified Linear Unit (ReLU).
#
•	MaxPooling2D-Schichten:
Die Schichten sind in Kombination mit Conv2D-Schichten sehr effektiv, weshalb sie im Anschluss an solch eine Schicht angewandt werden. Die MaxPooling2D-Schicht reduziert die Größe der Merkmals-Daten der Conv2D-Schicht. Das kann als eine Art Filter gesehen werden und bewirkt einerseits die Fokussierung auf wesentliche Merkmale. Andererseits führt es dazu, dass das Modell nicht auf spezifische Merkmale oder Rauschen trainiert. Außerdem unterstützt es auch die positionsunabhängige Erkennung von Merkmalen. Weiterer positiver Effekt dieser Schichten ist, dass die Reduzierung der Datenmenge die Berechnungslast verringert und das neuronale Netzwerk dadurch effizienter wird.
#
•	Flatten-Schicht:
Die Flatten-Schicht wandelt die Daten (mehrdimensional), die in diese Schicht gegeben werden, in einen eindimensionalen Vektor um. Dies ist für die Verarbeitung der nachfolgenden Dense-Schichten notwendig.
#
•	Dense-Schichten:
Jedes Neuron der Dense-Schichten (gewählt wurden z.B. 256 Neuronen) ist mit allen Neuronen aus den vorherigen Schichten verbunden. In den Dense-Schichten geschieht der eigentliche Lern-/ Trainingsprozess. Die Dense-Schicht knüpft sozusagen die Synapsen zwischen den Neuronen, um eine Analogie zur Biologie herzustellen. Dies geschieht durch das Anpassen der Gewichtungen und Parameter der Verbindungen einzelner Neuronen. Dadurch kann das Modell komplexe Zusammenhänge in den Daten erfassen. Die Dense-Schichten werden mit der Rectified Linear Unit (ReLU) aktiviert.
#
•	Dropout-Schicht:
Die Dropout-Schicht deaktiviert zufällig Neuronen. Dies hat den Effekt, dass das neuronale Netzwerk nicht zu stark auf die Trainingsdaten abgestimmt wird und lernt nicht zu stark auf bestimmte Neuronen angewiesen zu sein. Das soll die Robustheit gegenüber Ausfällen stärken und die Generalisierungsfähigkeit des Modells verbessern, sodass das Netz auch auf neuen Daten gut funktioniert. Mit dem Parameter stellt man die Wahrscheinlichkeit für ein Neuron ein, mit der das Neuron deaktiviert wird.
#
•	Dense-Schicht (Ausgangsschicht):
Die Ausgangs-Dense-Schicht hat nur 10 Neuronen die den 10 Klassen (Ziffern) entsprechen. Die Ausgangsschicht ist für die Klassifizierung zuständig und wird mit Softmax-Funktion aktiviert.


In [ ]:
# Modelldefinition
marvin = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

marvin.summary()

### Optimierung des Netzes
Für das Optimieren des Netzes während des Trainings wurden verschiedene Verfahren gewählt:
#
•	Als Optimierer wird der Adam-Optimizer verwendet. Warum habe ich mich für den Adam-Optimizer entschieden? 
#
Der Adam-Optimierer verwendet das Verfahren des stochastischen Gradienten-Abstiegs. Das bietet den Vorteil, dass eine adaptive Lernrate für jedes Gewicht möglich. Das soll die Konvergenz des Modells optimieren, da Gewichte mit kleinen Gradienten schneller konvergieren als Gewichte mit großen Gradienten. Auch ist der Adam-Optimierer für große Datensätze geeignet und steigert damit die Effizienz des neuronalen Netzwerks. Die Lernrate des Optimierers wurde auf 0,001 eingestellt, da die Lernrate somit nicht zu groß, aber auch nicht zu klein ist. Eine kleine Lernrate lässt das Modell langsam, jedoch stabil und mit konstantem Tempo konvergieren. Eine große Lernrate lässt das Modell schnell konvergieren, allerdings kann es passieren, dass das Modell dadurch überschießt oder oszilliert. Aus diesem Grund wurde mit der Lernrate 0,001 ein Kompromiss gewählt, der gute Ergebnisse erzielt.
#
•	Als Loss bzw. Verlustfunktion wurde CategoricalCrossentropy gewählt. Warum habe ich mich für diese Verlustfunktion entschieden?
#
Als Verlustfunktion wurde CategoricalCrossentropy gewählt, da die Verlustfunktion speziell für Klassifizierungen mit mehreren Klassen konzipiert ist. CategoricalCrossentropy ist effizient und bietet eine gute Softmax-Kompatibilität. Dadurch können Wahrscheinlichkeiten für Klassen interpretiert werden. Die zusätzliche Fokussierung auf die größte Wahrscheinlichkeit führt dazu, dass das Modell durch CategoricalCrossentropy in den Vorhersagen besser wird.
#
•	Als Metriken wurden, zusätzlich zur Standard-Metrik Accuracy, noch Precision und Recall verwendet. Warum habe ich mehrere Metriken verwendet und warum genau diese?
#
Mehrere Metriken sind sinnvoll, um die Leistung des Modells detaillierter zu bewerten. Auch können durch den Einsatz mehrerer Metriken verschiedene Aspekte, die die Leistung des Netzwerks bestimmen, betrachtet werden. Auch für die Anpassung der Hyperparameter und die Optimierung des Modells ist es hilfreich mehr Informationen über das Modell und seine Leistung zu haben.
Die Accuracy-Metrik gibt die Genauigkeit des Modells an. Hierfür wird nur der Prozentsatz der richtig klassifizierten Samples betrachtet.
Durch die Precision-Metrik kann die Präzision der Vorhersagen des Modells bewertet werden. Die Precision-Metrik misst den Prozentsatz der korrekt positiven Vorhersagen im Vergleich zu allen tatsächlich positiven Vorhersagen.
Die Recall-Metrik bezieht sich im Gegensatz zur Precision-Metrik auf die Sensitivität der Vorhersagen. Sie misst den Prozentsatz der korrekt positiven Vorhersagen im Vergleich zu allen tatsächlich positiven Instanzen.


In [ ]:
# Kompilieren des Modells
marvin.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# TensorBoard für die Überwachung
import datetime
import os
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


### Training des Netzes
Für das Training des Modells werden verschiedene Einstellungen gewählt.
#
•	Für das Training wurden 10 Epochen gewählt. Warum habe ich genau 10 Epochen gewählt?
#
Aufgrund der Wahl der Hyperparameter und der Einstellung der Lernrate auf 0,001 ist es notwendig, dass das Modell mehrere Epochen trainiert. Wird die Zahl der Epochen verringert, kann das Modell nicht konvergieren und man erreicht schlechte Ergebnisse in Genauigkeit, Präzision und Sensitivität. Ebenso ist eine deutlich höhere Anzahl an Epochen nicht förderlich für die Leistung des Netzes, da das Training über viele Epochen ressourcenaufwändig ist und viel Zeit beansprucht. Außerdem sinken Trainingsfortschritt und Optimierung, die pro Epoche erzielt werden mit jeder weiteren Epoche, d.h. das Kosten-Nutzen-Verhältnis wird von Epoche zu Epoche schlechter. Um also die Effizienz des Modells nicht zu reduzieren, wurde die Zahl der Epochen für das Training auf 10 eingestellt. Mit einer Epochenzahl von 15 kann ebenfalls gut gearbeitet werden, wie weitere Untersuchungen gezeigt haben.
#
•	Die Batch-Größe wurde für das Training auf 128 gesetzt. Warum habe ich diese Batch-Größe gewählt?
#
Die Batch-Größe ist mit 128 weder besonders groß noch besonders klein. Auch das liegt daran, dass hier eine Kompromiss-Lösung gewählt wurde, welche perfekt auf das Modell zugeschnitten ist. Da die Batch-Größe im mittleren Bereich liegt ist das Verhältnis aus Recheneffizienz und Speichereffizienz sehr gut. Ebenso ist die Batch-Größe auf das Zusammenspiel mit den Hyperparametern abgestimmt, sodass die Batchgröße bei gegebenen Hyperparametern eine schnellere Konvergenz und Anpassung der Gewichte pro Epoche ermöglicht. Aus diesem Grund wurde die Batch-Größe von 128 gewählt.


In [ ]:
# Modelltraining
marvin.fit(
    x_train,
    y_train_one_hot,
    epochs=10,
    batch_size=128,
    validation_data=(x_test, y_test_one_hot),
    callbacks=[tensorboard_callback]
)


In [ ]:
# Modell speichern
model_name = 'marvin.h5'
marvin.save(model_name, save_format='h5')
print('Success! You saved Marvin as: ', model_name)

model_name = 'marvin.h5' 
marvin_reloaded = tf.keras.models.load_model(model_name)

predictions = marvin_reloaded.predict([x_test])
predictions = np.argmax(predictions, axis=1)
pd.DataFrame(predictions)

numbers_to_display = 196
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(15, 15))
for plot_index in range(numbers_to_display):    
    predicted_label = predictions[plot_index]
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    color_map = 'Greens' if predicted_label == y_test[plot_index] else 'Reds'
    plt.subplot(num_cells, num_cells, plot_index + 1)
    plt.imshow(x_test_normalized[plot_index].reshape((28, 28)), cmap=color_map)
    plt.xlabel(predicted_label)

plt.subplots_adjust(hspace=1, wspace=0.5)
plt.show()

confusion_matrix = tf.math.confusion_matrix(y_test, predictions)
f, ax = plt.subplots(figsize=(9, 7))
sn.heatmap(
    confusion_matrix,
    annot=True,
    linewidths=.7,
    fmt="d",
    square=True,
    ax=ax,
    cmap="viridis",
)
plt.show()